Desenvolva um modelo preditivo de classificação para identificar cancelamentos e utilize métricas adequadas para argumentar a efetividade do modelo;

In [1]:
#import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
#from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
#from sklearn.naive_bayes import BernoulliNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

In [2]:
df = pd.read_csv("cancellation_prediction.csv")
df

,type,cancellation,days_between_booking_arrival,year_arrival_date,month_arrival_date,week_number_arrival_date,day_of_month_arrival_date,num_weekend_nights,num_workweek_nights,num_adults,...,num_previous_stays,reserved_room,changes_between_booking_arrival,deposit_policy,id_travel_agency_booking,id_person_booking,customer_type,avg_price,required_car_parking_spaces,total_of_special_requests
0,Fancy Hotel,0,342,2015,July,27,1,0,0,2,...,0,C,3,No Deposit,NaN,NaN,0,0.00,0,0
1,Fancy Hotel,0,737,2015,July,27,1,0,0,2,...,0,C,4,No Deposit,NaN,NaN,0,0.00,0,0
2,Fancy Hotel,0,7,2015,July,27,1,0,1,1,...,0,A,0,No Deposit,NaN,NaN,0,75.00,0,0
3,Fancy Hotel,0,13,2015,July,27,1,0,1,1,...,0,A,0,No Deposit,304.0,NaN,0,75.00,0,0
4,Fancy Hotel,0,14,2015,July,27,1,0,2,2,...,0,A,0,No Deposit,240.0,NaN,0,98.00,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119385,Hotel,0,23,2017,August,35,30,2,5,2,...,0,A,0,No Deposit,394.0,NaN,0,96.14,0,0
119386,Hotel,0,102,2017,August,35,31,2,5,3,...,0,E,0,No Deposit,9.0,NaN,0,225.43,0,2
119387,Hotel,0,34,2017,August,35,31,2,5,2,...,0,D,0,No Deposit,9.0,NaN,0,157.71,0,4
119388,Hotel,0,109,2017,August,35,31,2,5,2,...,0,A,0,No Deposit,89.0,NaN,0,104.40,0,0


In [3]:
#Removendo do df as colunas que serão tratadas
df_naive = df.drop(['type', 
                    'month_arrival_date', 
                    'breakfast', 
                    'country', 
                    'reserved_room', 
                    'deposit_policy'], axis = 1)
df_naive

,cancellation,days_between_booking_arrival,year_arrival_date,week_number_arrival_date,day_of_month_arrival_date,num_weekend_nights,num_workweek_nights,num_adults,num_children,num_babies,...,repeated_guest,num_previous_cancellations,num_previous_stays,changes_between_booking_arrival,id_travel_agency_booking,id_person_booking,customer_type,avg_price,required_car_parking_spaces,total_of_special_requests
0,0,342,2015,27,1,0,0,2,0.0,0,...,0,0,0,3,NaN,NaN,0,0.00,0,0
1,0,737,2015,27,1,0,0,2,0.0,0,...,0,0,0,4,NaN,NaN,0,0.00,0,0
2,0,7,2015,27,1,0,1,1,0.0,0,...,0,0,0,0,NaN,NaN,0,75.00,0,0
3,0,13,2015,27,1,0,1,1,0.0,0,...,0,0,0,0,304.0,NaN,0,75.00,0,0
4,0,14,2015,27,1,0,2,2,0.0,0,...,0,0,0,0,240.0,NaN,0,98.00,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119385,0,23,2017,35,30,2,5,2,0.0,0,...,0,0,0,0,394.0,NaN,0,96.14,0,0
119386,0,102,2017,35,31,2,5,3,0.0,0,...,0,0,0,0,9.0,NaN,0,225.43,0,2
119387,0,34,2017,35,31,2,5,2,0.0,0,...,0,0,0,0,9.0,NaN,0,157.71,0,4
119388,0,109,2017,35,31,2,5,2,0.0,0,...,0,0,0,0,89.0,NaN,0,104.40,0,0


In [4]:
#Transformando dtype object em int, de forma manual
type_hotel = df['type'].map({'Hotel': 0, 'Fancy Hotel' : 1})
month_arrival_date = df['month_arrival_date'].map({
                                                    'January' : 1, 
                                                    'February' : 2, 
                                                    'March' : 3, 
                                                    'April' : 4, 
                                                    'May' : 5, 
                                                    'June' : 6,
                                                    'July' : 7, 
                                                    'August' : 8, 
                                                    'September' : 9, 
                                                    'October' : 10, 
                                                    'November' : 11, 
                                                    'December' : 12,})
breakfast = df['breakfast'].map({True : 0, False : 1})
deposit_policy = df['deposit_policy'].map({'No Deposit': 0, 
                                                 'Refundable' : 1, 
                                                 'Non Refund' : 2})
transform_map = pd.concat([type_hotel, month_arrival_date, breakfast, deposit_policy], axis = 1)
transform_map

,type,month_arrival_date,breakfast,deposit_policy
0,1,7,0,0
1,1,7,0,0
2,1,7,0,0
3,1,7,0,0
4,1,7,0,0
...,...,...,...,...
119385,0,8,0,0
119386,0,8,0,0
119387,0,8,0,0
119388,0,8,0,0


In [5]:
#transformando através do get_dummies
Transform_dummies = pd.get_dummies(df.drop(['type', 
                                      'cancellation', 
                                      'days_between_booking_arrival',
                                     'year_arrival_date',
                                     'month_arrival_date',
                                     'week_number_arrival_date',
                                     'day_of_month_arrival_date',
                                     'num_weekend_nights',
                                     'num_workweek_nights',
                                     'num_adults',
                                     'num_children',
                                     'num_babies',
                                     'breakfast',
                                     'market_segment',
                                     'distribution_channel',
                                     'repeated_guest',
                                     'num_previous_cancellations',
                                     'num_previous_stays',
                                     'changes_between_booking_arrival',
                                     'deposit_policy',
                                     'id_travel_agency_booking',
                                     'id_person_booking',
                                     'customer_type',
                                     'avg_price',
                                     'required_car_parking_spaces',
                                     'total_of_special_requests'], axis = 1))
Transform_dummies

,country_ABW,country_AGO,country_AIA,country_ALB,country_AND,country_ARE,country_ARG,country_ARM,country_ASM,country_ATA,...,reserved_room_A,reserved_room_B,reserved_room_C,reserved_room_D,reserved_room_E,reserved_room_F,reserved_room_G,reserved_room_H,reserved_room_L,reserved_room_P
0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119385,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
119386,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
119387,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
119388,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [6]:
#juntando os dataframes tratados
df_final = pd.concat([df_naive, transform_map, Transform_dummies], axis = 1)
#removendo colunas 'id_travel_agency_booking' e 'id_person_booking', removendo nan.
df_final = df_final.drop(['id_travel_agency_booking', 'id_person_booking'], axis = 1).dropna()
df_final

,cancellation,days_between_booking_arrival,year_arrival_date,week_number_arrival_date,day_of_month_arrival_date,num_weekend_nights,num_workweek_nights,num_adults,num_children,num_babies,...,reserved_room_A,reserved_room_B,reserved_room_C,reserved_room_D,reserved_room_E,reserved_room_F,reserved_room_G,reserved_room_H,reserved_room_L,reserved_room_P
0,0,342,2015,27,1,0,0,2,0.0,0,...,0,0,1,0,0,0,0,0,0,0
1,0,737,2015,27,1,0,0,2,0.0,0,...,0,0,1,0,0,0,0,0,0,0
2,0,7,2015,27,1,0,1,1,0.0,0,...,1,0,0,0,0,0,0,0,0,0
3,0,13,2015,27,1,0,1,1,0.0,0,...,1,0,0,0,0,0,0,0,0,0
4,0,14,2015,27,1,0,2,2,0.0,0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119385,0,23,2017,35,30,2,5,2,0.0,0,...,1,0,0,0,0,0,0,0,0,0
119386,0,102,2017,35,31,2,5,3,0.0,0,...,0,0,0,0,1,0,0,0,0,0
119387,0,34,2017,35,31,2,5,2,0.0,0,...,0,0,0,1,0,0,0,0,0,0
119388,0,109,2017,35,31,2,5,2,0.0,0,...,1,0,0,0,0,0,0,0,0,0


#modelo knn

In [7]:
#Divisão em inputs e outputs
X = df_final.drop('cancellation', axis = 1)
y = df_final['cancellation']

In [8]:
#normalizando imouts 
norm = StandardScaler()
x_norm = norm.fit_transform(X)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(x_norm, y, test_size=0.3,random_state = 123)

treino teste

In [10]:
#decision tree
dtc= DecisionTreeClassifier(criterion='entropy', random_state=42)

In [11]:
dtc.fit(X_train, y_train)

DecisionTreeClassifier(criterion='entropy', random_state=42)

In [12]:
dtc.feature_importances_

array([1.21056782e-01, 3.27715703e-02, 5.57918081e-02, 6.14663027e-02,
       2.00966490e-02, 3.35217755e-02, 1.13353011e-02, 4.98614092e-03,
       3.78688017e-04, 9.18656959e-02, 6.36597119e-03, 1.94960365e-03,
       2.12516341e-02, 6.15471766e-03, 1.29307854e-02, 1.86568835e-02,
       9.07794782e-02, 3.33128036e-02, 3.67681346e-02, 7.99078660e-03,
       1.12652392e-02, 7.63577496e-03, 1.99718094e-01, 0.00000000e+00,
       1.53167052e-03, 0.00000000e+00, 0.00000000e+00, 1.04644792e-04,
       2.44505756e-04, 3.85070178e-04, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 4.88154206e-04, 1.06480567e-03,
       1.02651971e-04, 0.00000000e+00, 2.14795545e-03, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 1.02124710e-04, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 6.96659651e-05, 0.00000000e+00,
       2.20225578e-03, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       2.40614421e-03, 4.80402067e-05, 1.93636366e-03, 0.00000000e+00,
      

In [13]:
predict_dtc = dtc.predict(X_test)
predict_dtc

array([0, 0, 0, ..., 1, 1, 1], dtype=int64)

Validação

In [14]:
#mariz de confusão
print(confusion_matrix(y_test, predict_dtc)) 

[[19886  2775]
 [ 2694 10461]]


In [15]:
#acurácia
print(accuracy_score(y_test, predict_dtc)) 

0.8473028813937905


In [16]:
#precisão
print(precision_score(y_test, predict_dtc))

0.7903445149592022


In [17]:
#Recall
print(recall_score(y_test, predict_dtc)) 

0.7952109464082098


Ao realizar a validação cruzada do modelo de classificação, discuta sobre as diferenças entre utilizar uma separação entre treino e teste aleatória e uma separação temporal (Ex: treino em 2015 e 2016 e validação em 2017). Os resultados são diferentes? Qual o mais indicado?